In [2]:
! curl http://files.grouplens.org/datasets/movielens/ml-latest-small.zip -o ml-latest-small.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  955k  100  955k    0     0  4361k      0 --:--:-- --:--:-- --:--:-- 4381k


In [3]:
import zipfile
with zipfile.ZipFile('ml-latest-small.zip', 'r') as zip_ref:
    zip_ref.extractall('data')

In [4]:
# import the dataset
import pandas as pd
movies_df = pd.read_csv('data/ml-latest-small/movies.csv')
ratings_df = pd.read_csv('data/ml-latest-small/ratings.csv')

In [5]:
ratings_df.isnull().sum()

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

In [6]:
print('The dimensions of movies dataframe are:', movies_df.shape,'\nThe dimensions of ratings dataframe are:', ratings_df.shape)

The dimensions of movies dataframe are: (9742, 3) 
The dimensions of ratings dataframe are: (100836, 4)


In [7]:
# Take a look at movies_df
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [8]:
# Take a look at ratings_df
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [9]:
# Movie ID to movie name mapping
movie_names = movies_df.set_index('movieId')['title'].to_dict()
n_users = len(ratings_df.userId.unique())
n_items = len(ratings_df.movieId.unique())
print("Number of unique users:", n_users)
print("Number of unique movies:", n_items)
print("The full rating matrix will have:", n_users*n_items, 'elements.')
print('----------')
print("Number of ratings:", len(ratings_df))
print("Therefore: ", len(ratings_df) / (n_users*n_items) * 100, '% of the matrix is filled.')
print("We have an incredibly sparse matrix to work with here.")
print("And... as you can imagine, as the number of users and products grow, the number of elements will increase by n*2")
print("You are going to need a lot of memory to work with global scale... storing a full matrix in memory would be a challenge.")
print("One advantage here is that matrix factorization can realize the rating matrix implicitly, thus we don't need all the data")

Number of unique users: 610
Number of unique movies: 9724
The full rating matrix will have: 5931640 elements.
----------
Number of ratings: 100836
Therefore:  1.6999683055613624 % of the matrix is filled.
We have an incredibly sparse matrix to work with here.
And... as you can imagine, as the number of users and products grow, the number of elements will increase by n*2
You are going to need a lot of memory to work with global scale... storing a full matrix in memory would be a challenge.
One advantage here is that matrix factorization can realize the rating matrix implicitly, thus we don't need all the data


In [10]:
import torch
import numpy as np
from torch.autograd import Variable
from tqdm import tqdm_notebook as tqdm

class MatrixFactorization(torch.nn.Module):
    def __init__(self, n_users, n_items, n_factors=20):
        super().__init__()
        # create user embeddings
        self.user_factors = torch.nn.Embedding(n_users, n_factors) # think of this as a lookup table for the input.
        # create item embeddings
        self.item_factors = torch.nn.Embedding(n_items, n_factors) # think of this as a lookup table for the input.
        self.user_factors.weight.data.uniform_(0, 0.05)
        self.item_factors.weight.data.uniform_(0, 0.05)
        
    def forward(self, data):
        # matrix multiplication
        users, items = data[:,0], data[:,1]
        return (self.user_factors(users)*self.item_factors(items)).sum(1)
    # def forward(self, user, item):
    # 	# matrix multiplication
    #     return (self.user_factors(user)*self.item_factors(item)).sum(1)
    
    def predict(self, user, item):
        return self.forward(user, item)

In [11]:
# Creating the dataloader (necessary for PyTorch)
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader # package that helps transform your data to machine learning readiness

# Note: This isn't 'good' practice, in a MLops sense but we'll roll with this since the data is already loaded in memory.
class Loader(Dataset):
    def __init__(self):
        self.ratings = ratings_df.copy()
        
        # Extract all user IDs and movie IDs
        users = ratings_df.userId.unique()
        movies = ratings_df.movieId.unique()
        
        #--- Producing new continuous IDs for users and movies ---
        
        # Unique values : index
        self.userid2idx = {o:i for i,o in enumerate(users)}
        self.movieid2idx = {o:i for i,o in enumerate(movies)}
        
        # Obtained continuous ID for users and movies
        self.idx2userid = {i:o for o,i in self.userid2idx.items()}
        self.idx2movieid = {i:o for o,i in self.movieid2idx.items()}
        
        # return the id from the indexed values as noted in the lambda function down below.
        self.ratings.movieId = ratings_df.movieId.apply(lambda x: self.movieid2idx[x])
        self.ratings.userId = ratings_df.userId.apply(lambda x: self.userid2idx[x])
        
        
        self.x = self.ratings.drop(['rating', 'timestamp'], axis=1).values
        self.y = self.ratings['rating'].values
        self.x, self.y = torch.tensor(self.x), torch.tensor(self.y) # Transforms the data to tensors (ready for torch models.)

    def __getitem__(self, index):
        return (self.x[index], self.y[index])

    def __len__(self):
        return len(self.ratings)

In [12]:
num_epochs = 128
cuda = torch.cuda.is_available()

print("Is running on GPU:", cuda)

model = MatrixFactorization(n_users, n_items, n_factors=8)
print(model)
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)
# GPU enable if you have a GPU...
if cuda:
    model = model.cuda()

# MSE loss
loss_fn = torch.nn.MSELoss()

# ADAM optimizier
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Train data
train_set = Loader()
train_loader = DataLoader(train_set, 128, shuffle=True)

Is running on GPU: True
MatrixFactorization(
  (user_factors): Embedding(610, 8)
  (item_factors): Embedding(9724, 8)
)
user_factors.weight tensor([[1.6228e-02, 3.9171e-02, 4.2128e-02,  ..., 6.8354e-03, 4.8741e-02,
         5.5878e-03],
        [4.0266e-02, 4.8297e-02, 2.6058e-02,  ..., 3.1199e-02, 3.5918e-02,
         3.4840e-03],
        [2.5176e-03, 4.8367e-02, 1.4598e-02,  ..., 1.8356e-02, 1.6692e-02,
         9.8008e-03],
        ...,
        [2.5786e-02, 1.8316e-03, 2.7255e-03,  ..., 3.0395e-02, 7.4533e-05,
         4.8756e-02],
        [3.1013e-02, 8.5540e-03, 3.9906e-02,  ..., 2.8542e-02, 1.1774e-02,
         2.2980e-03],
        [1.2094e-03, 2.0475e-02, 1.8698e-03,  ..., 1.5306e-02, 4.9846e-02,
         2.5600e-03]])
item_factors.weight tensor([[0.0471, 0.0379, 0.0281,  ..., 0.0305, 0.0189, 0.0397],
        [0.0401, 0.0181, 0.0185,  ..., 0.0437, 0.0102, 0.0005],
        [0.0476, 0.0461, 0.0167,  ..., 0.0178, 0.0180, 0.0397],
        ...,
        [0.0276, 0.0102, 0.0037,  ..., 

In [13]:
for it in tqdm(range(num_epochs)):
    losses = []
    for x, y in train_loader:
         if cuda:
            x, y = x.cuda(), y.cuda()
            optimizer.zero_grad()
            outputs = model(x)
            loss = loss_fn(outputs.squeeze(), y.type(torch.float32))
            losses.append(loss.item())
            loss.backward()
            optimizer.step()
    print("iter #{}".format(it), "Loss:", sum(losses) / len(losses))

<ipython-input-13-dad152416852>:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for it in tqdm(range(num_epochs)):


  0%|          | 0/128 [00:00<?, ?it/s]

iter #0 Loss: 11.058398677612924
iter #1 Loss: 4.739438477809054
iter #2 Loss: 2.470549272250403
iter #3 Loss: 1.718456627148662
iter #4 Loss: 1.3444708386048447
iter #5 Loss: 1.1275538663724958
iter #6 Loss: 0.9913851194424073
iter #7 Loss: 0.9000405881761899
iter #8 Loss: 0.8371499494701473
iter #9 Loss: 0.7921968085057844
iter #10 Loss: 0.7592782526451924
iter #11 Loss: 0.7349451395309516
iter #12 Loss: 0.7159861410178509
iter #13 Loss: 0.7012820328930913
iter #14 Loss: 0.6904618207195083
iter #15 Loss: 0.6816771684337388
iter #16 Loss: 0.6751446443312059
iter #17 Loss: 0.6697196295569996
iter #18 Loss: 0.6658323556562971
iter #19 Loss: 0.6629154233989982
iter #20 Loss: 0.660488142542125
iter #21 Loss: 0.659062614409149
iter #22 Loss: 0.6574719655634788
iter #23 Loss: 0.6566728033541422
iter #24 Loss: 0.6555512687805946
iter #25 Loss: 0.6546919366656826
iter #26 Loss: 0.6538861556177212
iter #27 Loss: 0.6529695085689501
iter #28 Loss: 0.6515731512365608
iter #29 Loss: 0.650404661477

In [14]:
# By training the model, we will have tuned latent factors for movies and users.
c = 0
uw = 0
iw = 0 
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)
        if c == 0:
          uw = param.data
          c +=1
        else:
          iw = param.data
        #print('param_data', param_data)

user_factors.weight tensor([[ 1.2213,  0.3558,  1.5833,  ...,  1.5328,  0.8546,  1.8590],
        [ 2.3595,  1.9132,  0.8755,  ...,  1.0761,  0.1444,  0.2796],
        [-0.5044,  2.3478,  1.1034,  ...,  0.7290, -1.3774, -2.3098],
        ...,
        [ 0.2938,  1.5439, -0.0110,  ...,  0.7385,  1.4146,  0.2334],
        [ 0.5035,  0.9598,  0.9185,  ...,  0.6826,  1.5253,  1.4207],
        [ 1.3280,  1.1241,  0.6281,  ...,  0.5971,  1.3436,  0.5252]],
       device='cuda:0')
item_factors.weight tensor([[0.6076, 0.7611, 0.5599,  ..., 0.3501, 0.3830, 0.4259],
        [0.6079, 0.1175, 0.7743,  ..., 0.4028, 0.7541, 0.3726],
        [0.4703, 0.5040, 0.5293,  ..., 0.3775, 0.5748, 0.0828],
        ...,
        [0.3386, 0.3176, 0.3118,  ..., 0.3578, 0.3130, 0.3137],
        [0.3844, 0.4043, 0.4188,  ..., 0.4114, 0.3992, 0.3964],
        [0.4048, 0.3829, 0.4197,  ..., 0.4198, 0.3928, 0.4094]],
       device='cuda:0')


In [15]:
trained_movie_embeddings = model.item_factors.weight.data.cpu().numpy()

In [16]:
len(trained_movie_embeddings) # unique movie factor weights

9724

In [17]:
from sklearn.cluster import KMeans
# Fit the clusters based on the movie weights
kmeans = KMeans(n_clusters=10, random_state=0).fit(trained_movie_embeddings)

In [21]:
'''It can be seen here that the movies that are in the same cluster tend to have
similar genres. Also note that the algorithm is unfamiliar with the movie name
and only obtained the relationships by looking at the numbers representing how
users have responded to the movie selections.'''
for cluster in range(10):
  print("Cluster #{}".format(cluster))
  movs = []
  for movidx in np.where(kmeans.labels_ == cluster)[0]:
    movid = train_set.idx2movieid[movidx]
    rat_count = ratings_df.loc[ratings_df['movieId']==movid].count()[0]
    movs.append((movie_names[movid], rat_count))
  for mov in sorted(movs, key=lambda tup: tup[1], reverse=True)[:10]:
    print("\t", mov[0])

Cluster #0
	 Apollo 13 (1995)
	 Batman (1989)
	 Aladdin (1992)
	 True Lies (1994)
	 Men in Black (a.k.a. MIB) (1997)
	 Dances with Wolves (1990)
	 Mission: Impossible (1996)
	 Beauty and the Beast (1991)
	 Stargate (1994)
	 Titanic (1997)
Cluster #1
	 American Beauty (1999)
	 Back to the Future (1985)
	 Alien (1979)
	 Groundhog Day (1993)
	 Léon: The Professional (a.k.a. The Professional) (Léon) (1994)
	 One Flew Over the Cuckoo's Nest (1975)
	 Babe (1995)
	 Aliens (1986)
	 Blade Runner (1982)
	 E.T. the Extra-Terrestrial (1982)
Cluster #2
	 Forrest Gump (1994)
	 Shawshank Redemption, The (1994)
	 Silence of the Lambs, The (1991)
	 Matrix, The (1999)
	 Jurassic Park (1993)
	 Terminator 2: Judgment Day (1991)
	 Fight Club (1999)
	 Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)
	 Lord of the Rings: The Fellowship of the Ring, The (2001)
	 Star Wars: Episode VI - Return of the Jedi (1983)
Cluster #3
	 Toy Story (1995)
	 Independence Day (a.k.a. ID4) (1996)
